In [2]:
import os
import torch
import textwrap
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime

from datasets import load_dataset  # Hugging Face Datasets
from transformers import AlbertTokenizer, AlbertModel, AlbertConfig

In [3]:
# Load the WikiText-103 dataset (version 1)
wikitext = load_dataset("wikitext", "wikitext-103-v1")

In [4]:
def get_random_input(dataset, tokenizer):
    """
    Get a random input sequence from the dataset that meets minimum length requirements.

    dataset: HuggingFace dataset containing text data
    tokenizer: Tokenizer to convert text to model input format

    Q = why only use sequences longer than 300 tokens?
    A = because the model is large and the input sequence needs to be long enough to activate all the parameters

    Returns dictionary containing tokenized input with keys:
        - 'input_ids': Tensor of token IDs
        - 'attention_mask': Tensor of attention masks
    """
    n_train = len(dataset["train"])
    while True:
        # Randomly select an example from the dataset
        it = torch.randint(n_train, (1,)).item()
        text = dataset["train"][it]["text"]
        # Tokenize with padding and truncation
        ei = tokenizer(text, return_tensors="pt", truncation=True)
        # Only use sequences longer than 300 tokens
        if ei["input_ids"].shape[1] > 300:
            break
    return ei


def compute_correlations(hidden_states):
    """
    Compute pairwise correlations between token representations for each layer's hidden states.

    hidden_states (list):
        List of tensors containing hidden states from each layer of the model.
        Each tensor has shape (batch_size=1, sequence_length, hidden_dim)

    Returns list of tensors containing flattened correlation matrices for each layer.
    Each tensor contains the pairwise correlations between all tokens in that layer.
    The correlations are computed as cosine similarities between normalized token representations.
    """
    corrs = []
    for hs in hidden_states:
        # Remove batch dimension and create a copy without gradient tracking
        T = hs.squeeze(0).clone().detach().requires_grad_(False)
        # Normalize each token's representation to unit length for cosine similarity
        T = torch.nn.functional.normalize(T, dim=1)
        # Compute pairwise cosine similarities between all tokens
        T2 = torch.matmul(T, T.transpose(0, 1))
        corrs += [
            T2.flatten().cpu(),  # Flatten matrix and move to CPU for plotting
        ]
    return corrs

### Normal Model (24 layers)

In [5]:
al_tkz = AlbertTokenizer.from_pretrained("albert-xlarge-v2")
al_model = AlbertModel.from_pretrained("albert-xlarge-v2")
print(al_model.config.num_hidden_layers)

24


In [6]:
ei = get_random_input(wikitext, al_tkz)

print("- " * 20)
print("Random Input:")
decoded_text = al_tkz.batch_decode(
    ei["input_ids"],
    # skip_special_tokens=True
)
wrapped_text = [textwrap.fill(t, width=100) for t in decoded_text]
print("\n\n".join(wrapped_text))
print("- " * 20)

- - - - - - - - - - - - - - - - - - - - 
Random Input:
[CLS] callum blue portrays zod in season nine . zod is first mentioned in season five 's " arrival "
, when two of his disciples arrive on earth attempting to turn the planet into kryptonian utopia .
in the episode " solitude " , brainiac attempts to release him from his phantom zone prison , where
it is revealed that clark 's biological father jor <unk> -<unk> el placed zod 's spirit after
destroying his physical form . in the season five finale , zod is successfully transferred into lex
luthor 's body , after clark unknowingly releases him from the phantom zone . clark eventually pulls
zod 's spirit out of lex 's body using a kryptonian crystal of his father 's in the season six
premiere . in the season eight finale , the kryptonian purple orb , which was used in the season
seven finale to destroy the fortress of solitude and remove clark 's powers , appears at the luthor
mansion and releases zod in physical form . in the season 

In [ ]:
# get output features ... which contains last_hidden_state, pooler_output, hidden_states
# hidden_states is a list of tensors, each tensor has shape (batch_size=1, sequence_length, hidden_dim)
# last_hidden_state has shape (batch_size=1, sequence_length, hidden_dim)
# pooler_output has shape (batch_size=1, hidden_dim)

of = al_model(**ei, output_hidden_states=True)
correls = compute_correlations(of["hidden_states"])

In [ ]:
# Create a directory to save the plots
date_time = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"{date_time}_{al_model.config.num_hidden_layers}"
os.makedirs(f".plots/{folder_name}", exist_ok=True)

# Save the input text as txt file in the folder
with open(f".plots/{folder_name}/input_text.txt", "w") as f:
    f.write(decoded_text[0])

In [ ]:
# Create and save individual histogram plots for each layer's token correlations.
# Use adaptive binning based on data distribution and consistent x,y-axis scaling across all plots.

# determine global max density (y) value
max_density = 0
for data in correls:
    counts, bin_edges = np.histogram(data, bins=100, density=True)
    max_density = max(max_density, max(counts))

for i, data in enumerate(correls):
    IQR = np.percentile(data, 75) - np.percentile(data, 25)
    n = len(data)
    bin_width = 2 * IQR / n ** (1 / 3)
    bins = int((max(data) - min(data)) / bin_width)

    plt.figure()
    plt.hist(
        data,
        bins=bins,
        density=True,
        histtype="step",
        color="#3658bf",
        linewidth=1,
    )
    plt.title(f"Layer {i}", fontsize=12)
    plt.xlim(-0.3, 1.05)
    plt.ylim(0, max_density)

    plt.savefig(f"./plots/{folder_name}/histogram_layer_{i}.pdf")
    plt.close()

### Larger Model (48 layers)

In [13]:
alm2_config = AlbertConfig.from_pretrained(
    "albert-xlarge-v2", num_hidden_layers=48, num_attention_heads=1
)
almodel2 = AlbertModel.from_pretrained("albert-xlarge-v2", config=alm2_config)
print(almodel2.config.num_hidden_layers)

48
